In [1]:
import pandas as pd
import json


In [6]:
# # === Step 1: Load predictions ===
# svm_df = pd.read_excel(r"C:\Users\jefva\Documents\Master\Thesis_s2\Code\Baseline_Classifiers\Dataset3_no_context\collection_svm\SVM_test_predictions.xlsx")
# lr_df = pd.read_excel(r"C:\Users\jefva\Documents\Master\Thesis_s2\Code\Baseline_Classifiers\Dataset3_no_context\collection_logreg\logreg_test_predictions.xlsx")
# nb_df = pd.read_excel(r"C:\Users\jefva\Documents\Master\Thesis_s2\Code\Baseline_Classifiers\Dataset3_no_context\collection_NB\NaiveBayes_test_predictions.xlsx")
# rf_df = pd.read_excel(r"C:\Users\jefva\Documents\Master\Thesis_s2\Code\Baseline_Classifiers\Dataset3_no_context\collection_rf\RF_test_predictions.xlsx")
# xgb_df = pd.read_excel(r"C:\Users\jefva\Documents\Master\Thesis_s2\Code\Baseline_Classifiers\Dataset3_no_context\collection_xgb\xgb_test_predictions.xlsx")

# # === Step 2: Load label mappings ===
# with open(r"C:\Users\jefva\Documents\Master\Thesis_s2\results\final\mBERT\Run_2025-05-08_13-53\label_mappings.json", encoding="utf-8") as f:
#     mappings = json.load(f)

# theme_to_id = mappings["theme_to_id"]

# # === Step 3: Check if all questions are identical ===
# all_questions_equal = (
#     (svm_df["Question"] == lr_df["Question"]).all() and
#     (svm_df["Question"] == nb_df["Question"]).all() and
#     (svm_df["Question"] == rf_df["Question"]).all() and
#     (svm_df["Question"] == xgb_df["Question"]).all()
# )


# if not all_questions_equal:
#     raise ValueError("❌ The question texts are not identical across all prediction files!")

# print("✅ All question texts are identical across models.")

# # === Step 4: Convert predictions to theme IDs ===
# def map_preds(df):
#     return df["Predicted Label"].map(theme_to_id)

# combined_df = pd.DataFrame({
#     "question": svm_df["Question"],
#     "true_label": svm_df["True Label"].map(theme_to_id),  # assumes true label in same format
#     "LR": map_preds(lr_df),
#     "SVM": map_preds(svm_df),
#     "NB": map_preds(nb_df),
#     "RF": map_preds(rf_df),
#     "XGB": map_preds(xgb_df),
# })

# # === Step 5: Export to Excel ===
# output_path = r"C:\Users\jefva\Documents\Master\Thesis_s2\Code\Baseline_Classifiers\traditional_predictions_table.xlsx"
# combined_df.to_excel(output_path, index=False)
# print(f"✅ File saved to: {output_path}")


✅ All question texts are identical across models.
✅ File saved to: C:\Users\jefva\Documents\Master\Thesis_s2\Code\Baseline_Classifiers\traditional_predictions_table.xlsx


In [9]:
# import pandas as pd

# # === Step 1: Load the existing predictions file ===
# output_path = r"C:\Users\jefva\Documents\Master\Thesis_s2\Code\Baseline_Classifiers\traditional_predictions_table.xlsx"
# existing_df = pd.read_excel(output_path)

# # === Step 2: Load BERT model prediction files ===
# bert_df = pd.read_csv(r"C:\Users\jefva\Documents\Master\Thesis_s2\results\final\GroNLP\Run_2025-04-23_18-28\test_predictions.csv")
# robb_df = pd.read_csv(r"C:\Users\jefva\Documents\Master\Thesis_s2\results\final\roBERT\Run_2025-05-04_22-35\test_predictions.csv")
# mbert_df = pd.read_excel(r"C:\Users\jefva\Documents\Master\Thesis_s2\results\final\mBERT\Run_2025-05-08_13-53\predictions.xlsx")

# # === Step 3: Check if questions align ===
# if not (
#     (existing_df["question"] == bert_df["text"]).all() and
#     (existing_df["question"] == robb_df["text"]).all() and
#     (existing_df["question"] == mbert_df["clean_text"]).all()
# ):
#     raise ValueError("❌ Question texts do not match across all files.")

# print("✅ All question texts match.")

# # === Step 4: Add BERT model predictions as new columns ===
# existing_df["BERT"] = bert_df["predicted_label"]
# existing_df["RobBERT"] = robb_df["predicted_label"]
# existing_df["mBERT"] = mbert_df["mBERT"]

# # === Step 5: Save back to same Excel file ===
# existing_df.to_excel(output_path, index=False)
# print(f"✅ BERT predictions appended and saved to: {output_path}")



✅ All question texts match.
✅ BERT predictions appended and saved to: C:\Users\jefva\Documents\Master\Thesis_s2\Code\Baseline_Classifiers\traditional_predictions_table.xlsx


In [13]:
import pandas as pd
from scipy.stats import wilcoxon
from itertools import combinations

# === Load your data
df = pd.read_excel(r"C:\Users\jefva\Documents\Master\Thesis_s2\Code\Baseline_Classifiers\traditional_predictions_table.xlsx")  # Update path if needed

# === Define model columns
model_cols = ['LR', 'SVM', 'NB', 'RF', 'XGB', 'BERT', 'RobBERT', 'mBERT']

# === Convert predictions to binary correctness (1 = correct, 0 = incorrect)
correct_matrix = {
    model: (df[model] == df["True Label"]).astype(int) for model in model_cols
}
correct_df = pd.DataFrame(correct_matrix)

# === Wilcoxon tests with α = 0.05 (no correction)
alpha = 0.05
results = []

for model_a, model_b in combinations(model_cols, 2):
    try:
        stat, p_value = wilcoxon(correct_df[model_a], correct_df[model_b])
        significant = "Yes" if p_value < alpha else "No"
        results.append({
            "Model A": model_a,
            "Model B": model_b,
            "Wilcoxon Statistic": stat,
            "p-value": p_value,
            "Statistically Different (α = 0.05)": significant
        })
    except ValueError as e:
        results.append({
            "Model A": model_a,
            "Model B": model_b,
            "Wilcoxon Statistic": None,
            "p-value": None,
            "Statistically Different (α = 0.05)": "Error",
            "Error": str(e)
        })

# === Save results
results_df = pd.DataFrame(results)
results_df.to_excel("wilcoxon_pairwise_no_correction.xlsx", index=False)

# === Print some of the results
print(results_df)


    Model A  Model B  Wilcoxon Statistic        p-value  \
0        LR      SVM            202020.0   4.860995e-01   
1        LR       NB            711483.5   4.853962e-02   
2        LR       RF            696580.5   1.885173e-08   
3        LR      XGB            719352.0   2.022954e-02   
4        LR     BERT            445284.0  2.065536e-150   
5        LR  RobBERT            458398.5  8.753036e-144   
6        LR    mBERT            507448.5  1.045389e-125   
7       SVM       NB            601500.0   1.009409e-02   
8       SVM       RF            693420.0   2.636533e-07   
9       SVM      XGB            562504.5   5.263321e-02   
10      SVM     BERT            422052.5  1.509307e-148   
11      SVM  RobBERT            413553.0  5.158760e-144   
12      SVM    mBERT            430508.0  3.125149e-128   
13       NB       RF            716485.0   1.172775e-13   
14       NB      XGB            665464.0   1.615547e-05   
15       NB     BERT            607950.0  2.400090e-152 

In [20]:
import pandas as pd
from sklearn.metrics import classification_report
from scipy.stats import wilcoxon
from itertools import combinations

# === Load your data
df = pd.read_excel(r"C:\Users\jefva\Documents\Master\Thesis_s2\Code\Baseline_Classifiers\traditional_predictions_table.xlsx")

# === Define model columns
model_cols = ['LR', 'SVM', 'NB', 'RF', 'XGB', 'BERT', 'RobBERT', 'mBERT']
true_labels = df["True Label"]
classes = sorted(true_labels.unique())

# === Wilcoxon test on per-class F1-scores
alpha = 0.05
results = []

for model_a, model_b in combinations(model_cols, 2):
    # Compute classification reports
    report_a = classification_report(true_labels, df[model_a], labels=classes, output_dict=True, zero_division=0)
    report_b = classification_report(true_labels, df[model_b], labels=classes, output_dict=True, zero_division=0)
    
    # Extract F1-scores per class
    f1_a = [report_a[str(label)]["f1-score"] for label in classes]
    f1_b = [report_b[str(label)]["f1-score"] for label in classes]
    
    # Run Wilcoxon test
    try:
        stat, p_value = wilcoxon(f1_a, f1_b, alternative ='greater')
        significant = "Yes" if p_value < alpha else "No"
    except ValueError as e:
        stat, p_value = None, None
        significant = "Error"
    
    results.append({
        "Model A": model_a,
        "Model B": model_b,
        "Wilcoxon Statistic": stat,
        "p-value": p_value,
        "Statistically Different (α = 0.05)": significant
    })

# === Save results
results_df = pd.DataFrame(results)
results_df.to_excel("wilcoxon_per_class_f1_no_correction.xlsx", index=False)

# === Print some of the results
print(results_df)


    Model A  Model B  Wilcoxon Statistic   p-value  \
0        LR      SVM               106.0  0.196369   
1        LR       NB               134.0  0.017117   
2        LR       RF                17.0  0.999355   
3        LR      XGB                47.0  0.955116   
4        LR     BERT                 0.0  1.000000   
5        LR  RobBERT                 0.0  1.000000   
6        LR    mBERT                 0.0  1.000000   
7       SVM       NB                89.0  0.449287   
8       SVM       RF                23.0  0.997997   
9       SVM      XGB                38.0  0.982883   
10      SVM     BERT                 0.0  1.000000   
11      SVM  RobBERT                 0.0  1.000000   
12      SVM    mBERT                 0.0  1.000000   
13       NB       RF                 3.0  0.999749   
14       NB      XGB                16.0  0.997908   
15       NB     BERT                 0.0  1.000000   
16       NB  RobBERT                 0.0  1.000000   
17       NB    mBERT        

In [19]:
import pandas as pd
from sklearn.metrics import classification_report

# === Load your data
df = pd.read_excel(r"C:\Users\jefva\Documents\Master\Thesis_s2\Code\Baseline_Classifiers\traditional_predictions_table.xlsx")

# === Setup
true_labels = df["True Label"]
model_a = "BERT"
model_b = "RobBERT"
classes = sorted(true_labels.unique())

# === Generate classification reports
report_a = classification_report(true_labels, df[model_a], labels=classes, output_dict=True, zero_division=0)
report_b = classification_report(true_labels, df[model_b], labels=classes, output_dict=True, zero_division=0)

# === Optional: map ID to theme name (update if you have your own mapping)
theme_to_id = {
    "Brussel en de Vlaamse Rand": 0,
    "Energie": 1,
    "Milieu en Landbouw": 2,
    "Toerisme": 3,
    "Economie en Arbeid": 4,
    "Sport": 5,
    "Bestuur en Beleid": 6,
    "Justitie en Handhaving": 7,
    "Cultuur en Communicatie": 8,
    "Mobiliteit en Infrastructuur": 9,
    "Welzijn en Gezondheid": 10,
    "Begroting": 11,
    "Wonen": 12,
    "Onderwijs en Samenleving": 13,
    "Internationaal Beleid": 14,
    "Onroerend erfgoed": 15,
    "Financiën": 16,
    "Wetenschap en Innovatie": 17
}
id_to_theme = {v: k for k, v in theme_to_id.items()}
unique_themes = [id_to_theme[i] for i in classes]

# === Print F1-scores side by side
print("\nPer-class F1-score comparison:")
for theme, i in zip(unique_themes, classes):
    f1_a = report_a[str(i)]["f1-score"]
    f1_b = report_b[str(i)]["f1-score"]
    diff = f1_a - f1_b
    print(f"{theme:30s} BERT: {f1_a:.2f} | RobBERT: {f1_b:.2f} | Δ: {diff:+.2f}")



Per-class F1-score comparison:
Brussel en de Vlaamse Rand     BERT: 0.74 | RobBERT: 0.76 | Δ: -0.03
Energie                        BERT: 0.75 | RobBERT: 0.74 | Δ: +0.01
Milieu en Landbouw             BERT: 0.76 | RobBERT: 0.76 | Δ: -0.00
Toerisme                       BERT: 0.65 | RobBERT: 0.64 | Δ: +0.01
Economie en Arbeid             BERT: 0.77 | RobBERT: 0.76 | Δ: +0.01
Sport                          BERT: 0.64 | RobBERT: 0.62 | Δ: +0.02
Bestuur en Beleid              BERT: 0.73 | RobBERT: 0.74 | Δ: -0.01
Justitie en Handhaving         BERT: 0.68 | RobBERT: 0.65 | Δ: +0.03
Cultuur en Communicatie        BERT: 0.68 | RobBERT: 0.67 | Δ: +0.01
Mobiliteit en Infrastructuur   BERT: 0.87 | RobBERT: 0.87 | Δ: +0.00
Welzijn en Gezondheid          BERT: 0.74 | RobBERT: 0.75 | Δ: -0.01
Begroting                      BERT: 0.77 | RobBERT: 0.73 | Δ: +0.04
Wonen                          BERT: 0.71 | RobBERT: 0.67 | Δ: +0.04
Onderwijs en Samenleving       BERT: 0.73 | RobBERT: 0.73 | Δ: -0.00
In